This is a draft of how duckdb should be used in this project

In [43]:

from pathlib import Path

data_dir = Path('../../data/')

learn pathlib

In [44]:
print(data_dir)
print(sorted(data_dir.glob('*.csv')))
test_file = data_dir / 'C_LC.csv'
print(test_file)

../../data
[PosixPath('../../data/AA_C.csv'), PosixPath('../../data/C_AA.csv'), PosixPath('../../data/C_LC.csv'), PosixPath('../../data/C_TA.csv'), PosixPath('../../data/LC_C.csv'), PosixPath('../../data/TA_C.csv')]
../../data/C_LC.csv


### creating duckdb database of csv file

In [45]:
# in config file
chromosome_row = 2
pos_row = 3
start_row = 4
centromeres = {1: (29,30), 2: (7,9), 3: (26,29), 4: (6,10), 5: (22,26)}

In [46]:
import duckdb
rel = duckdb.read_csv(test_file, all_varchar=True)
#rel.show()
#duckdb.sql("SELECT COLUMNS('^\\d+_+\\d') FROM rel")
query = r"""
WITH cleaned_rel AS (
    SELECT * FROM rel
    WHERE Sample IS NOT NULL AND Sample != ''
),
long_data AS (
    SELECT 
        Sample,
        split_part(header, '_', 1)::INTEGER AS chr,
        split_part(header, '_', 2)::INTEGER AS pos,
        val AS genotype
    FROM cleaned_rel
    UNPIVOT (
        val FOR header IN (COLUMNS('^.+_+\d'))
    )
),
pivoted_data as (
    PIVOT long_data 
    ON genotype IN ('H', 'A', 'NA')
    GROUP BY chr, pos
)
SELECT 
    *,
    (H + A) AS valid_count,
    ROUND(H::FLOAT / NULLIF((H + A), 0), 3) AS h_ratio
FROM pivoted_data
ORDER BY chr, pos
"""

# Execute
tidy_data = duckdb.sql(query)
tidy_data.show()

┌───────┬───────┬───────┬───────┬───────┬─────────────┬─────────┐
│  chr  │  pos  │   H   │   A   │  NA   │ valid_count │ h_ratio │
│ int32 │ int32 │ int64 │ int64 │ int64 │    int64    │  float  │
├───────┼───────┼───────┼───────┼───────┼─────────────┼─────────┤
│     1 │     1 │    62 │    53 │     4 │         115 │   0.539 │
│     1 │     2 │    63 │    56 │     0 │         119 │   0.529 │
│     1 │     3 │    65 │    54 │     0 │         119 │   0.546 │
│     1 │     4 │    65 │    54 │     0 │         119 │   0.546 │
│     1 │     5 │    66 │    52 │     1 │         118 │   0.559 │
│     1 │     6 │    66 │    53 │     0 │         119 │   0.555 │
│     1 │     7 │    66 │    53 │     0 │         119 │   0.555 │
│     1 │     8 │    64 │    54 │     1 │         118 │   0.542 │
│     1 │     9 │    64 │    54 │     1 │         118 │   0.542 │
│     1 │    10 │    63 │    56 │     0 │         119 │   0.529 │
│     · │     · │     · │     · │     · │          ·  │     ·   │
│     · │ 

In [47]:
duckdb.sql("COPY tidy_data TO 'C_LC.test.csv'")